In [ ]:
# Brute Froce PARAFAC2 Decomposition of Shiraz Dataset


Note: this is a reconstruction of an experiment which was accidentally deleted.

Efforts of using CORCONDIA to estimate the rank of the dataset have failed. Thus we will attempt to 'brute force' a solution. This will provide insight into how the model changes over increasing rank.

In [ ]:
%reload_ext autoreload
%autoreload 2

# get the test data as two tables: metadata and a samplewise stacked img table
import logging

import duckdb as db

import polars as pl

from pca_analysis.definitions import DB_PATH_UV
from pca_analysis.parafac2_pipeline.parafac2_decomposition import (
    get_input_data,
)

from pca_analysis.get_sample_data import get_ids_by_varietal
import plotly.express as px

logger = logging.getLogger(__name__)

logger.setLevel(logging.DEBUG)


with db.connect(DB_PATH_UV) as conn:
    ids = get_ids_by_varietal("shiraz", conn)

input_data = get_input_data(DB_PATH_UV, ids)

input_data.filter_nm_tbl(pl.col("mins").le(4.55)).plot_2d_facet(
    wavelength=256,
    title="shiraz dataset @ 256",
    facet_col_wrap=4,
    template="plotly_dark",
)
